In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from keras import backend as K 
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential
from keras.models import Model 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from keras.applications import InceptionResNetV2 
! pip install split-folders

import split_folders


In [0]:
# training_data_folder = "/content/drive/My Drive/NN-ProjectC/Project_C1/Training/augmented_images"
train_data_dir = "/content/drive/My Drive/Neural_ProjectC1/ECE542_sp20_CompetitionData/TrainData/"
split_folders.ratio(training_data_folder, output = "train_val_noaug", seed=1337, ratio = (.7,.3))





Copying files: 0 files [00:00, ? files/s]

FileNotFoundError: ignored

In [0]:
train_data_dir = "train_val_noaug/train"
val_data_dir = "train_val_noaug/val"
num_train = 0
num_val = 0

for i in os.listdir(train_data_dir):
  if i.startswith("."):
    os.rmdir(os.path.join(train_data_dir, i))
  else:
    num_train += len(os.listdir(os.path.join(train_data_dir, i)))
  
for i in os.listdir(val_data_dir):
  if i.startswith("."):
    os.rmdir(os.path.join(val_data_dir, i))
  else:
    num_val += len(os.listdir(os.path.join(val_data_dir, i)))

img_width, img_height = 224, 224
# Check if the images are RGB and change the channels likewise
if K.image_data_format() == 'channels_first':
  input_shape= (3, img_width, img_height)
else:
  input_shape = (img_width, img_height, 3)

In [0]:
# CNN model
model = Sequential()
model.add(Conv2D(32,(5,5), input_shape = input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

#model.add(Conv2D(32,(5,5)))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64,(5,5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics = ['accuracy'])

In [0]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau

def get_callbacks():

   path_checkpoint ='checkpoint_keras'  
   log_dir='logs'
   
   callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                         monitor='val_loss',
                                         verbose=1,
                                         save_weights_only=False,
                                         save_best_only=True,
                                         mode='max',
                                         period=1)
   callback_early_stopping = EarlyStopping(monitor='val_loss',
                                           patience=5,
                                           verbose=1)
   callback_tensorboard = TensorBoard(log_dir=log_dir,
                                      histogram_freq=0,
                                      write_graph=False)
   callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                          factor=0.1,
                                          min_lr=1e-4,
                                          patience=3,
                                          verbose=1)

   callbacks = [callback_checkpoint, callback_tensorboard, callback_reduce_lr]

   return callbacks

In [0]:
batch_size = 64
epochs = 40
weight={0:1,1:3,2:4,3:4,4:6}
train_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = train_datagen.flow_from_directory(train_data_dir, 
                                                    target_size =(img_width, img_height), 
                                                    batch_size = batch_size, class_mode = 'categorical')
validation_generator = train_datagen.flow_from_directory( val_data_dir, 
                                                         target_size =(img_width, img_height), 
                                                         batch_size = batch_size, class_mode ='categorical') 

model.fit_generator(train_generator, 
                    steps_per_epoch = num_train // batch_size, 
                    epochs = epochs, 
                    validation_data = validation_generator, 
                    validation_steps = num_val// batch_size, 
                    callbacks = get_callbacks())

model.save_weights("data_aug.h5")